### SIMPLE_CHATBOT
<br>
In this guide, we will be applying LDA. we will need to download the amazon review dataset. let's get our libraries imported.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import string
import random
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import numpy as np

## AMAZON DATA

In [ ]:
# Opening the file
f = open("amazon_cells_labelled.txt", "r")

In [ ]:
data =[]
# Converting it to pandas dataframe
for line in f:
    review = line[:len(line) - 2]
    sentiment = line[len(line)-2]
    row = [review, sentiment]
    data.append(row)

amazon_data = pd.DataFrame(data, columns = ['reviews', 'sentiment'])

In [ ]:
amazon_data

,reviews,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.\t",1
2,Great for the jawbone.\t,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.\t,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.\t,0
998,The only thing that disappoint me is the infra...,0


In [ ]:
def cleaning_text(messages):
    """Clean and preprocess text data in a Pandas DataFrame column."""
    # Initializing an empty list
    clean_me = []
    for i in range(len(messages)):
        # create a WordNetLemmatizer object to lemmatize words
        lemm = WordNetLemmatizer()
        # Replacing any non-alphabetic characters with a space
        cleantext = re.sub('[^a-zA-Z]',' ',messages['reviews'][i])
        # Converting all characters to lowercase
        cleantext = cleantext.lower()
        # Splitting the text into a list of words
        cleantext = cleantext.split()
        # Removing all stopwords from the text from the NLTK library
        cleantext = [lemm.lemmatize(word) for word in cleantext if word not in stopwords.words('english')]
        # Joining the list of words
        cleantext = ' '.join(cleantext)
        # Adding the cleaned message to the list of cleaned messages
        clean_me.append(cleantext)
    return clean_me

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import warnings
nltk.download('omw-1.4')
amazon_d = cleaning_text(amazon_data)
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dhanushchandra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dhanushchandra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dhanushchandra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/dhanushchandra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/dhanushchandra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
def print_topics(topics, feature_names, sorting, topics_per_chunk, n_words):
    for i in range(0, len(topics), topics_per_chunk):
        these_topics = topics[i: i + topics_per_chunk]
        len_this_chunk = len(these_topics)
        print(these_topics)
        print(len_this_chunk)
        print(("topic {:<8}" * len_this_chunk).format(*these_topics))
        print(("-------- {0:<5}" * len_this_chunk).format(""))
        for i in range(n_words):
            try:
                top_words = [feature_names[sorting[topic, i]] for topic in these_topics]
                print(("{:<14}" * len_this_chunk).format(*top_words))
            except:
                break
        print("\n")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer(max_features=5000, max_df=.15)
X = v.fit_transform(amazon_d)

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, learning_method="batch", max_iter=25, random_state=0)

document_topics = lda.fit_transform(X)


sorting = np.argsort(lda.components_, axis=1)[:, ::-1]


feature_names = np.array(v.get_feature_names_out())

print_topics(topics=range(10), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)


range(0, 5)
5
topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
disappointed  good          quality       get           bad           
low           product       sound         money         quality       
impressed     price         poor          ear           pretty        
headset       great         battery       waste         fit           
item          buy           like          comfortable   one           
quality       excellent     time          time          call          
worth         happy         great         piece         end           
battery       battery       good          junk          best          
company       purchase      headset       make          really        
cool          sound         bluetooth     fit           thing         


range(5, 10)
5
topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      ----

In [ ]:
def preprocesstext(messages):
  # Creating a WordNetLemmatizer object to lemmatize words
  lemm = WordNetLemmatizer()
  # Replacing any non-alphabetic characters with a space
  cleantext = re.sub('[^a-zA-Z]',' ',messages)
  # Converting all characters to lowercase
  cleantext = cleantext.lower()
  # Splitting the text into a list of words
  cleantext = cleantext.split()
  # Removing stopwords from the text from the NLTK library
  cleantext = [lemm.lemmatize(word) for word in cleantext if word not in stopwords.words('english')]
  # Joinning the list of words back
  cleantext = ' '.join(cleantext)
  return cleantext

In [ ]:
import random
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TfidfVectorizer was used to vectorize the text data.
v = TfidfVectorizer()
reviews_vectorized_text = v.fit_transform(amazon_d)

# Establishing the chatbot function
def chatbot(user_input):
    # Clean and lemmatize the text before preprocessing user input.
    user_input_t = preprocesstext(user_input)
    user_input_vectorized_t = v.transform([user_input_t])
    # Calculating cosine similarities between user input and all reviews
    similarities_t = cosine_similarity(user_input_vectorized_t, reviews_vectorized_text)
    # Obtaining the index of similar reviews
    most_similar_index_t = similarities_t.argmax()
    # Obtaining the relevant review text and sentiment
    response_t = amazon_data.loc[most_similar_index_t, 'reviews']
    response_rate = amazon_data.loc[most_similar_index_t, 'sentiment']
    # Create a random response prefix based on the selected review's sentiment.
    if int(response_rate) ==0:
      response_prefix = random.choice(['Sorry to hear that...', 'That does not sound good...', 'We apologize for the inconvenience...'])
    else:
      response_prefix = random.choice(['Wow, thank you for your kind words...', 'We are thrilled to hear that...', 'Your feedback means a lot to us...'])
    # To generate the chatbot's response, concatenate the response prefix and review text.
    responses = f'{response_prefix}'
    return responses

# Testing the chatbot
print('Hi, I am chatbot and Welcome to the Amazon Reviews chatbot!')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    responses = chatbot(user_input)
    print(responses)


Hi, I am chatbot and Welcome to the Amazon Reviews chatbot!
> good
Wow, thank you for your kind words...
> bad
We apologize for the inconvenience...
> worst
We apologize for the inconvenience...
> disappointed
We apologize for the inconvenience...
> excellent
We are thrilled to hear that...
> very good
We are thrilled to hear that...
> quit


# CHATBOT

In [ ]:

paths = ['amazon_cells_labelled.txt', 'imdb_labelled.txt', 'yelp_labelled.txt']

data = []

# Loop through each file path
for path in paths:
    with open(path, "r") as f:
        for line in f:
            review = line[:-2]  # Assumes the last two characters are the sentiment and a newline
            sentiment = "neg" if line[-2] == "0" else "pos"
            data.append([review, sentiment])

# Convert list to DataFrame
df = pd.DataFrame(data, columns=['reviews', 'sentiment'])
df.head(1000)
df

,reviews,sentiment
0,So there is no way for me to plug it in here i...,neg
1,"Good case, Excellent value.\t",pos
2,Great for the jawbone.\t,pos
3,Tied to charger for conversations lasting more...,neg
4,The mic is great.\t,pos
...,...,...
2995,I think food should have flavor and texture an...,neg
2996,Appetite instantly gone.\t,neg
2997,Overall I was not impressed and would not go b...,neg
2998,"The whole experience was underwhelming, and I ...",neg


With our dataframe made, we now need to clean it before analyzing. Apply the `remove_punctuation()` and `remove_stopwords()` functions on our dataset to clean it.

In [ ]:
#Uncomment the line below if you need to download the stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')
# Instantiate the Porter Stemmer
stemmer = PorterStemmer()

def process_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords and stem the words
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop]

    # Join tokens back into a string
    return ' '.join(filtered_tokens)

df['reviews'] = df['reviews'].apply(process_text)
print(df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                            reviews sentiment
0                     way plug us unless go convert       neg
1                              good case excel valu       pos
2                                      great jawbon       pos
3  tie charger convers last 45 minutesmajor problem       neg
4                                         mic great       pos


We need to adjust our data slightly before using LDA. In the cell below, use the `CountVectorizer()` function. Then, use `fit_transform()` with `df['reviews']` as a parameter

In [ ]:
vect = CountVectorizer(max_features=5000, max_df=0.15)
X = vect.fit_transform(df['reviews'])

Using the `LatenDirichletAllocation()` function below, we want to pass it 10 components. You can adjust the max iterations for your local setup, or leave it as 25 if unsure.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, learning_method="batch", max_iter=25, random_state=0)
document_topics = lda.fit_transform(X)


And finally' let's see the results! Call the `print_topics()` function below, passing in `feature_names` and `sorting`.

In [ ]:
# Sorting the topics by the weight of each word
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# Getting the feature names from the vectorizer
feature_names = np.array(vect.get_feature_names_out())

# Function to print topics
def print_topics(topics, feature_names, sorting, topics_per_chunk, n_words):
    for i in range(0, len(topics), topics_per_chunk):
        these_topics = topics[i: i + topics_per_chunk]
        print("Topics:", *these_topics)
        print("----------------------")
        for j in range(n_words):
            try:
                print(" ".join(feature_names[sorting[these_topics, j]]))
            except Exception as e:
                print(e)
                pass
        print()

# Call the function
print_topics(topics=range(10), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)

Topics: 0 1 2 3 4
----------------------
disappoint bad get film love
dont servic use best wast
buy like ear ive good
film act pretti one time
custom even headset ever ever
movi noth fantast movi money
one good see think one
place better end seen place
nice get like never worst
realli movi say like realli

Topics: 5 6 7 8 9
----------------------
suck great phone place sound
film food work back price
charact good recommend go good
poor movi great movi qualiti
realli look product wont movi
one time well good great
plot terribl excel 10 worth
got servic good film definit
everi film would make thing
get batteri case easi use



In [ ]:
reviews = ""
for review in df["reviews"]:
    reviews += review
print(reviews)

way plug us unless go convertgood case excel valugreat jawbontie charger convers last 45 minutesmajor problemmic greatjiggl plug get line right get decent volumsever dozen sever hundr contact imagin fun send one onerazr ownery mustneedless say wast moneywast money timesound qualiti greatimpress go origin batteri extend batteritwo seper mere 5 ft start notic excess static garbl sound headsetgood qualiti thoughdesign odd ear clip comforthighli recommend one blue tooth phoneadvis everyon foolfar goodwork greatclick place way make wonder long mechan would lastwent motorola websit follow direct could get pairbought use kindl fire absolut lovecommerci misleadyet run new batteri two bar that three day without chargbought mother problem batterigreat pocket pc phone combinive own phone 7 month say best mobil phone ivedidnt think instruct provid helppeopl couldnt hear talk pull earphon talk phonedoesnt hold chargsimpl littl phone use breakag unacceptproduct ideal peopl like whose ear sensitunus 

In [ ]:
# Preprocess function definition
def preprocess_text(messages):
    lemm = WordNetLemmatizer()
    cleantext = re.sub('[^a-zA-Z]', ' ', messages)
    cleantext = cleantext.lower()
    cleantext = cleantext.split()
    stop_words = set(stopwords.words('english'))
    cleantext = [lemm.lemmatize(word) for word in cleantext if word not in stop_words]
    cleantext = ' '.join(cleantext)
    return cleantext


# TF-IDF Vectorization of the clean reviews
v = TfidfVectorizer()
reviews_vectorized_text = v.fit_transform(df['reviews'])

# Chatbot function
def chatbot(user_input):
    try:
        user_input_t = preprocess_text(user_input)
        user_input_vectorized_t = v.transform([user_input_t])
        similarities_t = cosine_similarity(user_input_vectorized_t, reviews_vectorized_text)
        most_similar_index_t = similarities_t.argmax()
        response_text = df['reviews'].iloc[most_similar_index_t]
        response_sentiment = df['sentiment'].iloc[most_similar_index_t]
        if response_sentiment == "neg":
            response_prefix = random.choice(['Sorry to hear that...', 'That does not sound good...', 'We apologize for the inconvenience...'])
        else:
            response_prefix = random.choice(['Wow, thank you for your kind words...', 'We are thrilled to hear that...', 'Your feedback means a lot to us...'])
        response = f'{response_prefix} {response_text}'
    except Exception as e:
        response = f"Error processing your input: {str(e)}"
    return response

# Chatbot interaction setup
print('Hi, I am chatbot and Welcome to the Reviews chatbot!')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    response = chatbot(user_input)
    print(response)

Hi, I am chatbot and Welcome to the Reviews chatbot!
> bad
We apologize for the inconvenience... Bad characters, bad story and bad acting.  	
> good
We are thrilled to hear that... Good food , good service .	
> worst
Sorry to hear that... Worst ever.	
> excellent
Wow, thank you for your kind words... Excellent!.	
> good food
Your feedback means a lot to us... The food is good.	
> bad food
Sorry to hear that... Bad characters, bad story and bad acting.  	
> bad acting
That does not sound good... The acting was bad.  	
> good product
Your feedback means a lot to us... Really good product.	
> must visit
We are thrilled to hear that... An absolute must visit!	
> best
Wow, thank you for your kind words... Best headset ever!!!.	
> thank you
Your feedback means a lot to us... Thank you for such great service.	
> quit
